In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q scann

In [ ]:
import numpy as np 
import os
import zipfile
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tqdm import tqdm
from typing import Dict, Text

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TensorFlow Recommender version: {}'.format(tfrs.__version__))
print('TensorFlow Raking')
print('TensorFlow ScaNN')

In [ ]:
TRAIN_DIR = '/kaggle/input/otto-pp-train/otto_pp_train'
ITEM_DIR = '/kaggle/input/otto-preprocessed-items-1'
OUTPUT_DIR = '/kaggle/working/'
CHECK_DIR = '/kaggle/working/chkpt1'

# MODEL_DIR = f'{OUTPUT_DIR}/serving_model'
# MODEL_DIR_SCAN = f'{MODEL_DIR}/ScaNN_Model'
# MODEL_DIR_NORM = f'{MODEL_DIR}/Norm_Model'

# !mkdir -p {MODEL_DIR}
# !mkdir -p {MODEL_DIR_SCAN}
# !mkdir -p {MODEL_DIR_NORM}


In [ ]:
gpus = tf.config.list_physical_devices("GPU")
for gpu in gpus:
    print(gpu)

In [ ]:
if gpus:
    # Create 2 virtual GPUs with 1GB memory each
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=4096),
             tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
        
        logical_gpus = tf.config.list_logical_devices("GPU")
        print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
    
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print('error')
        print(e)

strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
# Create a description of the features.
feature_description = {
    'item_A': tf.io.FixedLenFeature([], tf.int64),
    'item_B': tf.io.FixedLenFeature([], tf.int64),
    'rating': tf.io.FixedLenFeature([], tf.float32),
    'ts': tf.io.FixedLenFeature([], tf.int64),
}

In [ ]:
def convert2string(element):
    return {"item_A" : tf.strings.as_string(element['item_A']),
            "item_B" : tf.strings.as_string(element['item_B']),
            "rating" : element['rating'],
            "ts"     : element['ts']}

In [ ]:
with strategy.scope():
    dataset = tf.data.Dataset.list_files(f'{TRAIN_DIR}/pptrain_chunk_*.tfrecord')
    dataset = dataset.interleave(lambda x: tf.data.TFRecordDataset(x, compression_type='ZLIB'), block_length=1024, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x: tf.io.parse_single_example(x, feature_description),num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size=1024, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(convert2string)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
%%time
with strategy.scope():
    items_dataset = tf.data.Dataset.load(ITEM_DIR)
    items_dataset = items_dataset.batch(batch_size=1024,num_parallel_calls=tf.data.AUTOTUNE)
    itemlist_unique = np.unique(np.concatenate(list(items_dataset)))

# A multitask Model

There are two critical parts to multi-task recommenders:

* They optimize for two or more objectives, and so have two or more losses.
* They share variables between the tasks, allowing for transfer learning.
In this tutorial, we will define our models as before, but instead of having a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

In [ ]:
with strategy.scope():
    class item2itemModel(tfrs.models.Model):
        def __init__(self) -> None:
            super().__init__()
            embedding_dimension = 32
            self.item_A_model = tf.keras.Sequential([
                tf.keras.layers.StringLookup(
                    vocabulary=itemlist_unique, mask_token=None),
                tf.keras.layers.Embedding(len(itemlist_unique) + 1, embedding_dimension)
                ])

            self.item_B_model = tf.keras.Sequential([
                tf.keras.layers.StringLookup(
                    vocabulary=itemlist_unique, mask_token=None),
                tf.keras.layers.Embedding(len(itemlist_unique) + 1, embedding_dimension)
                ])

            self.rating_model = tf.keras.Sequential([
                tf.keras.layers.Dense(256, activation="relu"),
                tf.keras.layers.Dense(128, activation="relu"),
                tf.keras.layers.Dense(1),
                ])

            self.retrieval_task = tfrs.tasks.Retrieval(
                metrics=tfrs.metrics.FactorizedTopK(
                    candidates=items_dataset.map(self.item_B_model),
                    )
                )

            self.rating_task = tfrs.tasks.Ranking(
                loss=tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM),
                metrics=[tf.keras.metrics.RootMeanSquaredError()],
                )

        def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
            item_A_embeddings = self.item_A_model(features["item_A"])
            item_B_embeddings = self.item_B_model(features["item_B"])
            predicted_ratings = self.rating_model((tf.concat([item_A_embeddings, item_B_embeddings], axis=1)))
            
            return (item_A_embeddings, item_B_embeddings, predicted_ratings)

        def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
            label_ratings = features.pop("rating")
            item_A_embeddings, item_B_embeddings, predicted_ratings = self(features)
            rating_loss = self.rating_task(labels=label_ratings, predictions=predicted_ratings)
            retrieval_loss = self.retrieval_task(item_A_embeddings, item_B_embeddings)

            return (retrieval_loss + rating_loss)       

In [ ]:
with strategy.scope():
    class item2itemModelBase(tf.keras.Model):
        def __init__(self) -> None:
            super().__init__()
            embedding_dimension = 32
            self.item_A_model = tf.keras.Sequential([
                tf.keras.layers.StringLookup(
                    vocabulary=itemlist_unique, mask_token=None),
                tf.keras.layers.Embedding(len(itemlist_unique) + 1, embedding_dimension)
                ])

            self.item_B_model = tf.keras.Sequential([
                tf.keras.layers.StringLookup(
                    vocabulary=itemlist_unique, mask_token=None),
                tf.keras.layers.Embedding(len(itemlist_unique) + 1, embedding_dimension)
                ])

            self.rating_model = tf.keras.Sequential([
                tf.keras.layers.Dense(256, activation="relu"),
                tf.keras.layers.Dense(128, activation="relu"),
                tf.keras.layers.Dense(1),
                ])

            self.retrieval_task = tfrs.tasks.Retrieval(
                metrics=tfrs.metrics.FactorizedTopK(
                    candidates=items_dataset.map(self.item_B_model),
                    )
                )

            self.rating_task = tfrs.tasks.Ranking(
                loss=tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM),
                metrics=[tf.keras.metrics.RootMeanSquaredError()],
                )

        def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
            item_A_embeddings = self.item_A_model(features["item_A"])
            item_B_embeddings = self.item_B_model(features["item_B"])
            predicted_ratings = self.rating_model((tf.concat([item_A_embeddings, item_B_embeddings], axis=1)))
            return (item_A_embeddings, item_B_embeddings, predicted_ratings)

        def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
            with tf.GradientTape() as tape:
                label_ratings = features.pop("rating")
                item_A_embeddings, item_B_embeddings, predicted_ratings = self(features)
                rating_loss = self.rating_task(labels=label_ratings, predictions=predicted_ratings, compute_metrics=False)
                retrieval_loss = self.retrieval_task(item_A_embeddings, item_B_embeddings, compute_metrics=False)
                regularization_loss = sum(self.losses)
                total_loss = rating_loss + retrieval_loss + regularization_loss
            gradients = tape.gradient(total_loss, self.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

            metrics={}
            metrics["loss"] = rating_loss + retrieval_loss
            metrics["regularization_loss"] = regularization_loss
            metrics["total_loss"] = total_loss
            return metrics
            
        def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
            with tf.GradientTape() as tape:
                label_ratings = features.pop("rating")
                item_A_embeddings, item_B_embeddings, predicted_ratings = self(features)
                rating_loss = self.rating_task(labels=label_ratings, predictions=predicted_ratings, compute_metrics=False)
                retrieval_loss = self.retrieval_task(item_A_embeddings, item_B_embeddings, compute_metrics=False)
                regularization_loss = sum(self.losses)
                total_loss = rating_loss + retrieval_loss + regularization_loss

#             metrics = {metric.name: metric.result() for metric in self.metrics}
            metrics={}
            metrics["loss"] = retrieval_loss
            metrics["regularization_loss"] = regularization_loss
            metrics["total_loss"] = total_loss
            return metrics

In [ ]:
with strategy.scope():
    model = item2itemModelBase()
    model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:

with strategy.scope():
#     train_dataset = dataset.shuffle(1000000, seed=42, reshuffle_each_iteration=False)
    cached_train = dataset.cache()

In [ ]:
with strategy.scope():
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='/kaggle/working/chkpt1',
        save_weights_only=True
    )

In [ ]:
with strategy.scope():
    model.fit(cached_train, epochs=3, callbacks=[model_checkpoint_callback])

In [ ]:
# with strategy.scope():
#     test_dataset = dataset.shuffle(1000, seed=42, reshuffle_each_iteration=False).take(100)
#     cached_test = test_dataset.shuffle(100).cache()
#     metrics = model.evaluate(cached_test)

In [ ]:
#     scann_index = tfrs.layers.factorized_top_k.ScaNN(model.item_A_model, k=20)
#     scann_index.index_from_dataset(
#       tf.data.Dataset.zip((items_dataset, items_dataset.map(model.item_B_model)))
#     )

In [ ]:
#     # Get recommendations.
#     _, titles = scann_index(tf.constant(["1585682"]))
#     print(f"Recommendations for user 1585682: {titles[0, :10]}")

In [ ]:
# # Save the index.
# tf.saved_model.save(
#       scann_index,
#       MODEL_DIR_SCAN,
#       options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"])
#   )

In [ ]:
# model.retrieval_task = tfrs.tasks.Retrieval()  # Removes the metrics.
# model.compile()
# model.save(MODEL_DIR_NORM)

In [ ]:
# !zip -r trainer.zip '/kaggle/working/serving_model'

In [ ]:
!zip -r chkpt_1.zip '/kaggle/working/'